## From BNC to Ngram - clarification 
There seems to be a general sense of confusion regarding this homeword assignment. I would try to provide a more comprehensive instruction with some hints to handle some of the problems encountered during class.

## Objective
The end goal of this assignment is to generate the rank and rank ratio between BNC and clang8 for all the bigrams with the format "adj. accident"


#### BNC Data:  
https://drive.google.com/file/d/1mKX1DLHDIqKph4e4k1MnYOV3iWtvT7-E/view?usp=sharing

#### Cleaned Lang 8 Data:
https://drive.google.com/file/d/11wxKJr-VpmrHZ-MR41i4E097uY-T9okH/view?usp=sharing


### A. Processing BNC Data
### 1.1 Extract lines containing id, title, classcode, keywords, sentences from each BNC parts (ABCDEFGHJK)

using grep / egrep to match regular expression and extract relavent data  

Reference
https://www.twblogs.net/a/5d26d705bd9eee1e5c84509d

In [ ]:
# extract data from raw data
! egrep -o -h \
'(<idno type="bnc">.*?</idno>|<title>.*?</title>|<classCode.*?</classCode>|<keywords>.*?</keywords>|<s n=".*?">|<w c5=".*?" hw=".*?" pos=".*?">.*?</w>|<c c5=".*?">.*?</c>|</s>|<p>|</p>)' \
Texts/*/*/A*.xml | sed 's/></>\n</g' > BNC.A.txt

egrep -o -h  Texts/*/*/A*.xml > BNC.A.txt  237.80s user 1.29s system 99% cpu 4:00.15 total


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**NOTE:** From the lab, it seems most systems output  

\<s n="1"\>\<w c5="NN1" hw="factsheet" pos="SUBST"\>FACTSHEET \</w\>\<w c5="DTQ" hw="what" pos="PRON"\>WHAT \</w\>\<w c5="VBZ" hw="be" pos="VERB"\>IS \</w\>\<w c5="NN1" hw="aids" pos="SUBST"\>AIDS\</w\>...\</s\>  
  
as one line. For line_to_token() to work, please sepereate each word (\<w\> ~ \</w\>) into one line.

In [ ]:
! head BNC.A.txt

<title> [ACET factsheets &amp; newsletters]. Sample containing about 6688 words of miscellanea (domain: social science) </title>
<idno type="bnc">A00</idno>
<title> [ACET factsheets &amp; newsletters]. </title>
<classCode scheme="DLEE">W nonAc: medicine</classCode>
<keywords><term> Health </term><term> Sex </term></keywords>
<s n="1">
<w c5="NN1" hw="factsheet" pos="SUBST">FACTSHEET </w>
<w c5="DTQ" hw="what" pos="PRON">WHAT </w>
<w c5="VBZ" hw="be" pos="VERB">IS </w>
<w c5="NN1" hw="aids" pos="SUBST">AIDS</w>


 ### 2. Convert sentences to bigram (for all sections A to K, no I)
 After you extract all the BNC data (BNC.A.txt, BNC.B.txt, BNC.C.txt ...), you need to process xml into tokens and bigrams.
 
 ### 2.1 Convert line to word tokens

In [ ]:
import re
from pprint import pprint

def line_to_token(line):
    if line.startswith('<s'):
        return ('<s> ', '<s>', '<s>') 
    elif line.startswith('</s'):
        return ('</s>', '</s>', '</s>') 
    elif line.startswith('<w'):
        # <w c5="VVN" hw="discount" pos="VERB">discounted </w>
        match = re.findall('<w c5="(.*?)" hw="(.*?)" pos=".*?">(.*?)</w>', line)
        return (match[0][2].strip(), match[0][0].upper(), match[0][1]) #  , tag, word
    elif line.startswith('<c'):
        match = re.findall('<c c5="PUN">(.*?)</c>', line)
        if not match:
            return '??? line'
        return (match[0], match[0], match[0])

def tokens_to_bigram(tokens):
    result = []
    for i in range(len(tokens)-1):
        if i == 1:
            word2tag2lemma2 = [tokens[i][j].lower()+' '+tokens[i+1][j] for j in range(3)]
        else:
            word2tag2lemma2 = [tokens[i][j]+' '+tokens[i+1][j] for j in range(3)]
        if word2tag2lemma2[0][0].isalpha() or word2tag2lemma2[0][0] == '<': 
            result = result + [ '\t'.join(word2tag2lemma2) ]
    return result

### 2.2 Convert token stream to bigram stream

In [ ]:
def word_to_bigram(wordfile, bigramfile):
    
    def Batch_to_ngram(batch, fileout):        
        with open(wordfile.format(batch),'r',encoding="utf-8") as filein:
            lines = filein.readlines()
            for i, line in enumerate(lines):
                if line.startswith('<s'):
                    sent_start = i
                elif line.startswith('</s'):
                    sentence = [line.strip() for line in lines[sent_start:i+1]]
                    tokens = [line_to_token(line) for line in sentence ]
                    bigram = tokens_to_bigram(tokens)
                    print('\n'.join(bigram), file=fileout)
    
    with open(bigramfile, 'w',encoding="utf-8") as fileout:
        for batch in 'A':
            Batch_to_ngram(batch, fileout)
                
word_to_bigram('BNC.{0}.txt', 'BNC.2w.txt')

0
5
24
[('<s> ', '<s>', '<s>'),
 ('This', 'DT0', 'this'),
 ('virus', 'NN1', 'virus'),
 ('affects', 'VVZ', 'affect'),
 ('the', 'AT0', 'the'),
 ('body', 'NN1', 'body'),
 ("'s", 'POS', "'s"),
 ('defence', 'NN1', 'defence'),
 ('system', 'NN1', 'system'),
 ('so', 'AV0', 'so'),
 ('that', 'CJT', 'that'),
 ('it', 'PNP', 'it'),
 ('can', 'VM0', 'can'),
 ('not', 'XX0', 'not'),
 ('fight', 'VVI', 'fight'),
 ('infection', 'NN1', 'infection'),
 ('</s>', '</s>', '</s>')]
------------------------
41
46
55
62
[('<s> ', '<s>', '<s>'),
 ('from', 'PRP', 'from'),
 ('an', 'AT0', 'an'),
 ('infected', 'AJ0', 'infected'),
 ('mother', 'NN1', 'mother'),
 ('to', 'PRP', 'to'),
 ('her', 'DPS', 'she'),
 ('baby', 'NN1', 'baby'),
 ('</s>', '</s>', '</s>')]
------------------------
71
77
[('<s> ', '<s>', '<s>'),
 ('giving', 'VVG', 'give'),
 ('blood/mosquito', 'NN1', 'blood/mosquito'),
 ('bites/toilet', 'UNC', 'bites/toilet'),
 ('seats/kissing/from', 'UNC', 'seats/kissing/from'),
 ('normal', 'AJ0', 'normal'),
 ('day-to-d

 ('basically', 'AV0', 'basically'),
 ('managed', 'VVD', 'manage'),
 ('district', 'NN1', 'district'),
 ('nurses', 'NN2', 'nurse'),
 ('health', 'NN1', 'health'),
 ('visitors', 'NN2', 'visitor'),
 ('school', 'NN1', 'school'),
 ('and', 'CJC', 'and'),
 ('family', 'NN1', 'family'),
 ('planning', 'NN1-VVG', 'planning'),
 ('nurses', 'NN2', 'nurse'),
 ('he', 'PNP', 'he'),
 ('said', 'VVD', 'say'),
 ('</s>', '</s>', '</s>')]
------------------------
5016
[('<s> ', '<s>', '<s>'),
 ('Kay', 'NP0', 'kay'),
 ('an', 'AT0', 'an'),
 ('ACET', 'NP0', 'acet'),
 ('nurse', 'NN1', 'nurse'),
 ('visits', 'VVZ', 'visit'),
 ('a', 'AT0', 'a'),
 ('client', 'NN1', 'client'),
 ('to', 'TO0', 'to'),
 ('administer', 'VVI', 'administer'),
 ('Petamadine', 'NP0-NN1', 'petamadine'),
 ('to', 'TO0', 'to'),
 ('help', 'VVI', 'help'),
 ('prevent', 'VVI', 'prevent'),
 ('him', 'PNP', 'he'),
 ('developing', 'AJ0-VVG', 'developing'),
 ('pneumocystis', 'NN1', 'pneumocystis'),
 ('corinii', 'NN2', 'corinii'),
 ('pneumonia', 'NN1', 'pneu

 ('with', 'PRP', 'with'),
 ('each', 'DT0', 'each'),
 ('other', 'NN1', 'other'),
 ('and', 'CJC', 'and'),
 ('to', 'TO0', 'to'),
 ('understand', 'VVI', 'understand'),
 ('each', 'DT0', 'each'),
 ('other', 'NN1', 'other'),
 ('</s>', '</s>', '</s>')]
------------------------
8600
8606
8627
8641
8651
8667
[('<s> ', '<s>', '<s>'),
 ('You', 'PNP', 'you'),
 ('lose', 'VVB', 'lose'),
 ('weight', 'NN1', 'weight'),
 ('you', 'PNP', 'you'),
 ('feel', 'VVB', 'feel'),
 ('very', 'AV0', 'very'),
 ('weak', 'AJ0', 'weak'),
 ('and', 'CJC', 'and'),
 ('pass', 'VVB-NN1', 'pass'),
 ('motions', 'NN2', 'motion'),
 ('up', 'AVP', 'up'),
 ('to', 'PRP', 'to'),
 ('10', 'CRD', '10'),
 ('times', 'NN2', 'time'),
 ('a', 'AT0', 'a'),
 ('day', 'NN1', 'day'),
 ('</s>', '</s>', '</s>')]
------------------------
8685
8696
8700
8712
8723
8740
[('<s> ', '<s>', '<s>'),
 ('People', 'NN0', 'people'),
 ('with', 'PRP', 'with'),
 ('AIDS', 'NN2', 'aid'),
 ('are', 'VBB', 'be'),
 ('often', 'AV0', 'often'),
 ('well', 'AJ0', 'well'),
 ('for

 ('assuming', 'VVG', 'assume'),
 ('that', 'CJT', 'that'),
 ('you', 'PNP', 'you'),
 ('are', 'VBB', 'be'),
 ('a', 'AT0', 'a'),
 ('taxpayer', 'NN1', 'taxpayer'),
 ('you', 'PNP', 'you'),
 ('can', 'VM0', 'can'),
 ('enable', 'VVI', 'enable'),
 ('ACET', 'NP0', 'acet'),
 ('to', 'TO0', 'to'),
 ('get', 'VVI', 'get'),
 ('the', 'AT0', 'the'),
 ('benefit', 'NN1', 'benefit'),
 ('of', 'PRF', 'of'),
 ('tax', 'NN1', 'tax'),
 ('relief', 'NN1', 'relief'),
 ('by', 'PRP', 'by'),
 ('the', 'AT0', 'the'),
 ('same', 'DT0', 'same'),
 ('Deposited', 'VVD', 'deposit'),
 ('Covenant', 'NN1-VVB', 'covenant'),
 ('arrangement', 'NN1', 'arrangement'),
 ('</s>', '</s>', '</s>')]
------------------------
12746
12747
12749
12760
12798
12829
12861
12919
12921
12930
12953
12974
[('<s> ', '<s>', '<s>'),
 ('Similarly', 'AV0', 'similarly'),
 ('if', 'CJS', 'if'),
 ('your', 'DPS', 'you'),
 ('spouse', 'NN1', 'spouse'),
 ('pays', 'VVZ', 'pay'),
 ('income', 'NN1', 'income'),
 ('tax', 'NN1', 'tax'),
 ('at', 'PRP', 'at'),
 ('the', 'AT

[('<s> ', '<s>', '<s>'),
 ('I', 'PNP', 'i'),
 ('feel', 'VVB', 'feel'),
 ('that', 'CJT', 'that'),
 ('this', 'DT0', 'this'),
 ('is', 'VBZ', 'be'),
 ('reflected', 'VVN', 'reflect'),
 ('not', 'XX0', 'not'),
 ('just', 'AV0', 'just'),
 ('in', 'PRP', 'in'),
 ('the', 'AT0', 'the'),
 ('increased', 'AJ0', 'increased'),
 ('knowledge', 'NN1', 'knowledge'),
 ('of', 'PRF', 'of'),
 ('the', 'AT0', 'the'),
 ('students', 'NN2', 'student'),
 ('but', 'CJC', 'but'),
 ('also', 'AV0', 'also'),
 ('in', 'PRP', 'in'),
 ('a', 'AT0', 'a'),
 ('change', 'NN1', 'change'),
 ('of', 'PRF', 'of'),
 ('attitude', 'NN1', 'attitude'),
 ('and', 'CJC', 'and'),
 ('a', 'AT0', 'a'),
 ('few', 'DT0', 'few'),
 ('greater', 'AJC', 'great'),
 ('degrees', 'NN2', 'degree'),
 ('of', 'PRF', 'of'),
 ('empathy', 'NN1', 'empathy'),
 ('</s>', '</s>', '</s>')]
------------------------
15508
15513
[('<s> ', '<s>', '<s>'),
 ('If', 'CJS', 'if'),
 ('there', 'EX0', 'there'),
 ('is', 'VBZ', 'be'),
 ('to', 'TO0', 'to'),
 ('be', 'VBI', 'be'),
 ('an', 

...

1386932
1386955
1386989
1387012
1387056
1387095
1387131
1387180
1387206
1387229
1387237
1387286
1387315
[('<s> ', '<s>', '<s>'),
 ('It', 'PNP', 'it'),
 ("'s", 'VBZ', 'be'),
 ('true', 'AJ0-AV0', 'true'),
 ('the', 'AT0', 'the'),
 ('horse', 'NN1', 'horse'),
 ('was', 'VBD', 'be'),
 ('100', 'CRD', '100'),
 ('per', 'UNC', 'per'),
 ('cent', 'UNC', 'cent'),
 ('sound', 'NN1', 'sound'),
 ('five', 'CRD', 'five'),
 ('minutes', 'NN2', 'minute'),
 ('afterwards', 'AV0', 'afterwards'),
 ('but', 'CJC', 'but'),
 ('there', 'EX0', 'there'),
 ('was', 'VBD', 'be'),
 ('definitely', 'AV0', 'definitely'),
 ('something', 'PNI', 'something'),
 ('wrong', 'AV0-AJ0', 'wrong'),
 ('down', 'AVP', 'down'),
 ('at', 'PRP', 'at'),
 ('the', 'AT0', 'the'),
 ('start', 'NN1', 'start'),
 ('and', 'CJC', 'and'),
 ('Willie', 'NP0', 'willie'),
 ('had', 'VHD', 'have'),
 ('advice', 'NN1', 'advice'),
 ('from', 'PRP', 'from'),
 ('the', 'AT0', 'the'),
 ('starter', 'NN1', 'starter'),
 ('and', 'CJC', 'and'),
 ('a', 'AT0', 'a'),
 ('vet', 

1398327
1398340
1398368
1398381
1398404
1398421
1398436
1398464
1398476
1398494
1398509
[('<s> ', '<s>', '<s>'),
 ('But', 'CJC', 'but'),
 ('yesterday', 'AV0', 'yesterday'),
 ("'s", 'POS', "'s"),
 ('report', 'NN1', 'report'),
 ('from', 'PRP', 'from'),
 ('the', 'AT0', 'the'),
 ('US', 'NP0', 'us'),
 ('Purchasing', 'NN1-VVG', 'purchasing'),
 ('Managers', 'NN2', 'manager'),
 ('which', 'DTQ', 'which'),
 ('put', 'VVB', 'put'),
 ('the', 'AT0', 'the'),
 ('index', 'NN1', 'index'),
 ('of', 'PRF', 'of'),
 ('export', 'NN1', 'export'),
 ('performance', 'NN1', 'performance'),
 ('at', 'PRP', 'at'),
 ('its', 'DPS', 'it'),
 ('highest', 'AJS', 'high'),
 ('level', 'NN1', 'level'),
 ('since', 'PRP-CJS', 'since'),
 ('April', 'NP0', 'april'),
 ('is', 'VBZ', 'be'),
 ('another', 'DT0', 'another'),
 ('red', 'AJ0-NN1', 'red'),
 ('rag', 'NN1', 'rag'),
 ('to', 'PRP', 'to'),
 ('dollar', 'NN1', 'dollar'),
 ('bulls', 'NN2', 'bull'),
 ('looking', 'VVG', 'look'),
 ('for', 'PRP', 'for'),
 ('a', 'AT0', 'a'),
 ('narrower'

1410422
1410466
1410488
1410514
1410554
1410587
1410614
[('<s> ', '<s>', '<s>'),
 ('I', 'PNP', 'i'),
 ('still', 'AV0', 'still'),
 ('intend', 'VVB', 'intend'),
 ('to', 'TO0', 'to'),
 ('train', 'VVI', 'train'),
 ('as', 'PRP', 'as'),
 ('a', 'AT0', 'a'),
 ('doctor', 'NN1', 'doctor'),
 ('because', 'CJS', 'because'),
 ('I', 'PNP', 'i'),
 ('feel', 'VVB', 'feel'),
 ('there', 'EX0', 'there'),
 ('is', 'VBZ', 'be'),
 ('no', 'AT0', 'no'),
 ('more', 'AV0', 'more'),
 ('interesting', 'AJ0', 'interesting'),
 ('and', 'CJC', 'and'),
 ('worthwhile', 'AJ0', 'worthwhile'),
 ('way', 'NN1', 'way'),
 ('to', 'TO0', 'to'),
 ('spend', 'VVI', 'spend'),
 ('my', 'DPS', 'i'),
 ('life', 'NN1', 'life'),
 ('but', 'CJC', 'but'),
 ('how', 'AVQ', 'how'),
 ('many', 'DT0', 'many'),
 ('other', 'AJ0', 'other'),
 ('people', 'NN0', 'people'),
 ('will', 'VM0', 'will'),
 ('be', 'VBI', 'be'),
 ('able', 'AJ0', 'able'),
 ('to', 'TO0', 'to'),
 ('or', 'CJC', 'or'),
 ('want', 'VVB', 'want'),
 ('to', 'TO0', 'to'),
 ('in', 'AVP', 'in'),


1428560
1428595
1428612
1428623
1428639
1428669
1428685
1428710
1428723
1428757
1428788
1428826
1428873
1428893
1428907
[('<s> ', '<s>', '<s>'),
 ('As', 'CJS', 'as'),
 ('they', 'PNP', 'they'),
 ('do', 'VDB', 'do'),
 ('so', 'AV0', 'so'),
 ('yet', 'AV0', 'yet'),
 ('another', 'DT0', 'another'),
 ('Asian', 'AJ0', 'asian'),
 ('city', 'NN1', 'city'),
 ('ironically', 'AV0', 'ironically'),
 ('in', 'PRP', 'in'),
 ('the', 'AT0', 'the'),
 ('circumstances', 'NN2', 'circumstance'),
 ('the', 'AT0', 'the'),
 ('most', 'AV0', 'most'),
 ('Thatcherite', 'AJ0', 'thatcherite'),
 ('of', 'PRF', 'of'),
 ('them', 'PNP', 'they'),
 ('all', 'DT0', 'all'),
 ('will', 'VM0', 'will'),
 ('be', 'VBI', 'be'),
 ('dying', 'VVG-AJ0', 'die'),
 ('</s>', '</s>', '</s>')]
------------------------
1428930
[('<s> ', '<s>', '<s>'),
 ('Hong', 'NP0', 'hong'),
 ('Kong', 'NP0', 'kong'),
 ('Special', 'AJ0', 'special'),
 ('Report', 'NN1', 'report'),
 ('Slogans', 'NN2', 'slogan'),
 ('</s>', '</s>', '</s>')]
------------------------
1428

KeyboardInterrupt: 

### 3 Sort and count bigram (word1 word2 \<tab\> count) 

In [ ]:
#1 BNC.2w.txt ==> BNC.2w.c.txt
# We sort the bigrams and count identical bigrams
# NOTE: BNC.2w.c.txt should be considerably smaller than BNC.2w.txt
# NOTE: Since we only care about "adj. accident", you can filter BNC.2w.c.txt into a much
#       smaller file by extracting bigrams that fit "adj. accident"

! time sort BNC.2w.txt | uniq -c | \
awk '{ gsub(/^[ ]*/, ""); print }' | awk '{print substr($0, index($0, " ")+1) "\t" $1}' > BNC.2w.c.txt

sort: No such file or directory
sort BNC.B.2w.txt  0.00s user 0.00s system 54% cpu 0.005 total
uniq -c  0.00s user 0.00s system 49% cpu 0.004 total
awk '{ gsub(/^[ ]*/, ""); print }'  0.00s user 0.00s system 49% cpu 0.004 total
awk '{print substr($0, index($0, " ")+1) "\t" $1}'  0.00s user 0.00s system 52% cpu 0.004 total
egrep -v '\t1$' > BNC.B.2w.c2+.txt  0.00s user 0.00s system 56% cpu 0.003 total


In [ ]:
# Example data format for BNC.2w.c.txt
# bigram       pos      lemmas          count

! egrep '^(big|serious|fatal) accident\t' BNC.2w.c.txt
! egrep 'accident\s.*AJ' BNC.2W.c.txt > adj_accident.txt

big accident	AJ0 NN1	big accident	3
fatal accident	AJ0 NN1	fatal accident	82
fatal accident	aj0 NN1	fatal accident	3
serious accident	AJ0 NN1	serious accident	61


### B. Processing clang8 Data
During the lab, we found out that the lang8 dataset we provided last week did not contain enough "adj. accident" bigrams for this assignment.  
Please process the clang8 data provided above and extract bigrams.  
**NOTE:** We are only intereted in the rank / rank ratio for  "adj. accident" bigrams. Please extract bigrams of "adj. accident" from both BNC and clang8 first, then calculate the rank and rank ratio. 

Target output format: 
https://drive.google.com/file/d/1xM46aaDIeu4Z0FkikGOcmDoq7u2O47tY/view?usp=sharing

Demo time sign up for Lab 3:
https://docs.google.com/spreadsheets/d/1OKbXhcv6E3FEQDPnbHEHEeHvpxv01jxugMP7WwnKqKw/edit?usp=sharing

For Demo, please print out all the "adj. accident" bigrams in descending rank ratio order.



In [4]:
import os
import re
import string
from pprint import pprint

In [2]:
def tokenize(text):
    """
    Input:
    "This is an example.'

    Sample output: 
    ['this', 'is', 'an', 'example', '.']
    """  
    #### [ TODO ] transform text to lower case
    lowerText = text.lower()
    pureText = re.sub(r'[.,"\'-?:!;]', '', re.sub(r'[0-9]+', '', lowerText))
    #### [ TODO ] seperate the words by white space
    splitText = pureText.split()
    return splitText
    
from collections import Counter

def calculate_frequency(tokens):
    """
    Input:
    ['this', 'is', 'an', 'example', ...]

    Sample output: 
    {
        'the': 79809, 
        'project': 288,
        ...
    }
    """
    #### [ TODO ] 
    topToken = Counter(tokens).most_common()
    di = dict(topToken)
    return di
   
def ranking(tokens):
    rank = 0
    previous_number = 0
    for element in tokens:
        rank += 1
        tokens[element] = rank
    return tokens

def get_ngram(tokens,n):
    ngram_result = []
    ngram_list = [tokens[i:i+n] for i in range(0,len(tokens)-1)]
    for element in ngram_list:
        ngram_result.append(' '.join(element[0 : n]))
    return ngram_result

    """
    Input:
    ['this', 'is', 'an', 'example', ...]

    Sample output: 
    ['this is', 'is an', 'an example', ...]
    """
    #### [TODO] 
    

In [5]:
f = open('/content/clang8.txt', 'r',encoding="utf-8")
fileRead = f.read()
#### [ TODO ] generate lang8 unigrams and calculate document frequency of unigram in lang8
clang_unigram = tokenize(fileRead)
clang_unigram_counter = calculate_frequency(clang_unigram)
f.close()

In [7]:
clang_bigram = get_ngram(clang_unigram,2)
clang_bigram_counter = calculate_frequency(clang_bigram)
clang_bigram_Rank = ranking(clang_bigram_counter)
print(clang_bigram_Rank)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

